In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
ruta_base = "anid/datos-separados/"
ruta_final = "anid/finales/"

## Unificar datos

In [3]:
xls1 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_1982_2002.xlsx")
xls2 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_2003_2022.xlsx")
xls3 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_2023_2032.xlsx")

xls_unificado = pd.concat( [xls1, xls2, xls3] )
xls_unificado.to_excel(f"{ruta_final}proyectos-unificados-anid.xlsx")

## Primer filtro

In [4]:
tmp_array = xls_unificado.to_numpy()
final_list = []

for row in tmp_array:
    if type(row[22]) != float and type(row[8]) != float:
        #Solo buscar temas antarticos
        if ("ANTARC" in row[22].upper() or "ANTART" in row[22].upper() or "ANTARC" in row[8].upper() or "ANTART" in row[8].upper()):
            final_list.append( row )
        else:
            pass

In [5]:
dt = pd.DataFrame( np.array(final_list), columns=['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO', 'MONEDA', 'PALABRAS_CLAVES'])
dt.to_excel(f"{ruta_final}proyectos-primer-filtro.xlsx")

## agregar filtro al la base general

In [21]:
xls1 = pd.read_excel(f"BDH_PROYECTOS_1982_2002.xlsx")
xls2 = pd.read_excel(f"BDH_PROYECTOS_2003_2022.xlsx")
xls3 = pd.read_excel(f"BDH_PROYECTOS_2023_2032.xlsx")

xls = pd.concat( [xls1, xls2, xls3] )

In [35]:
xls_antartico = pd.read_excel(f"filtrado-amano.xlsx")['CODIGO_PROYECTO'].tolist()

In [36]:
todos_codigos_proyecto = xls['CODIGO_PROYECTO'].tolist()
lista_booleana = []

for codigo in todos_codigos_proyecto:
    if codigo in xls_antartico:
        print("+1")
        lista_booleana.append('SI')
    else:
        lista_booleana.append('NO')

+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1


In [37]:
tmp_array = xls.to_numpy().T.tolist()
tmp_array.append( lista_booleana )
tmp_array = np.array( tmp_array ).T

In [38]:
dt = pd.DataFrame( tmp_array, columns=['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO', 'MONEDA', 'PALABRAS_CLAVES', 'ANTARTIC'])
dt.to_excel("datos-totales-columna-nueva.xlsx")

In [35]:
xls = pd.read_excel(f"BDH_PROYECTOS_2003_2022.xlsx")
xls1 = xls[ xls['AGNO_CONCURSO'] >= 2019 ]

xls2 = pd.read_excel(f"BDH_PROYECTOS_2023_2032.xlsx")

xls = pd.concat([xls1, xls2])
xls.head()

,N,CODIGO_PROYECTO,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO,MONEDA,PALABRAS_CLAVES
23508,33177,1190302,PROYECTOS DE INVESTIGACION,FONDECYT,REGULAR,FONDECYT REGULAR 2019,2019.0,2019,PRODUCTION OF HMF IN AN IONIC LIQUID/ORGANIC B...,INGENIERIA Y TECNOLOGIA,...,PERSONA NATURAL,JULIO ROMERO FIGUEROA,HOMBRE,UNIVERSIDAD DE SANTIAGO DE CHILE,RM,REGIÓN METROPOLITANA DE SANTIAGO,167475,NaN,Miles de pesos (M$),5-HYDROXYMETHYLFURFURAL;IONIC LIQUID;DENSE CAR...
23509,33178,MEC80190041,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - MEC,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,2019.0,2019,PRYM-TYURIN VARIETIES II,CIENCIAS NATURALES,...,PERSONA JURIDICA,RUBI RODRIGUEZ,MUJER,UNIVERSIDAD DE LA FRONTERA,SUR,REGIÓN DE LA ARAUCANÍA,11100,NaN,Miles de pesos (M$),ABELIAN VARIETIES;GROUP ACTIONS;ENDOMORPHISMS
23510,33179,1190991,PROYECTOS DE INVESTIGACION,FONDECYT,REGULAR,FONDECYT REGULAR 2019,2019.0,2019,THE GEOMETRY OF ABELIAN VARIETIES WITH GROUP A...,CIENCIAS NATURALES,...,PERSONA NATURAL,RUBI RODRIGUEZ MORENO,MUJER,UNIVERSIDAD DE LA FRONTERA,SUR,REGIÓN DE LA ARAUCANÍA,83640,NaN,Miles de pesos (M$),ABELIAN VARIETIES;SYMMETRIES;RIEMANN SURFACES
23511,33180,ACT192073,CENTROS E INVESTIGACION ASOCIATIVA,PIA,ANILLOS DE INVESTIGACION EN CIENCIAS Y TECNOLOGIA,CONCURSO ANILLOS DE INVESTIGACION EN CIENCIA Y...,2019.0,2019,PLANT ABIOTIC STRESS FOR A SUSTAINABLE AGRICUL...,MULTIDISCIPLINARIO,...,PERSONA JURIDICA,CLAUDIA STANGE KLEIN,MUJER,UNIVERSIDAD DE CHILE,MULTIREGIONAL,MULTIREGIONAL,450000,NaN,Miles de pesos (M$),ABIOTIC STRESS;CLIMATE-CHANGE;SUSTAINABLE AGRI...
23512,33181,11190818,PROYECTOS DE INVESTIGACION,FONDECYT,INICIACION,CONCURSO DE PROYECTOS FONDECYT DE INICIACION E...,2019.0,2019,ABUSO SEXUAL EN INSTITUCIONES RELIGIOSAS DEL N...,CIENCIAS SOCIALES,...,PERSONA NATURAL,CRISTIAN GUSTAVO PINTO CORTEZ,HOMBRE,UNIVERSIDAD DE TARAPACA,NORTE,REGIÓN DE ARICA Y PARINACOTA,39402,NaN,Miles de pesos (M$),ABUSO SEXUAL;INSTITUCION RELIGIOSA;VICTIMAS


In [38]:
tmp_array = xls.to_numpy()
n = 0
final_list = []
for row in tmp_array:
    if type(row[22+n])==str:
        if "ANTARC" in row[22+n] or "ANTARC" in row[8+n]:
            #print("test")
            print(row[1+n], row[3+n], row[14+n], row[8+n])
        else:
            pass

3190489 FONDECYT MARCELO IVAN VERA ESCALONA ANTARCTIC INVASION IN A WARMER AND CONNECTED WORLD: CLIMATE CHANGE, MARITIME TRANSPORT AND RAFTING AS THREATENING FACTORS OF ANTARCTIC MARINE BIODIVERSITY
DFG190004 FONDEQUIP RAUL R. CORDERO ASSESSING THE VARIABILITY OF THE COMPOSITION OF THE SOUTHERN HEMISPHERE UPPER TROPOSPHERE AND LOWER STRATOSPHERE AT THE NORTHERN TIP OF THE ANTARCTIC PENINSULA.
REDES190078 PCI PATRICIO RAMOS CONSOLIDATION OF A SCIENTIFIC NETWORK TO STUDY MOLECULAR AND PHYSIOLOGICAL MECHANISMS OF MICROBE–PLANT INTERACTIONS IN AN ANTARCTIC BIOME CHALLENGED BY CLIMATE CHANGE
1191932 FONDECYT RAUL CORDERO CARRASCO ATMOSPHERIC RADIATION MEASUREMENTS ON KING GEORGE ISLAND (SOUTHERN OCEAN / ANTARCTIC PENINSULA)
1190230 FONDECYT GARY SHAFFER MODELLING THE INTERACTIVE EVOLUTIONS OF CLIMATE, CARBON CYCLING AND THE ANTARCTIC ICE SHEET ACROSS THE EOCENE-OLIGOCENE TRANSITION USING THE DCESS EARTH SYSTEM AND ANTARCTIC ICE SHEET MODELS
11190802 FONDECYT RODOLFO  RONDON SALLAN BIG EFFEC